In [1]:
from cutList import cutList
from SetConfiguration import SetConfiguration
import ROOT
from ROOT import gROOT,TFile,TH1F,gDirectory,TChain,RDataFrame,SetOwnership
#ROOT.ROOT.EnableImplicitMT()
from copy import deepcopy
import os
import json
import numpy as np
import math
import time
import sys
import logging
#Dask imports
import dask
from dask.distributed import LocalCluster, Client
from dask.distributed.client import _get_global_client
#%matplotlib notebook
from dask_kubernetes.operator import KubeCluster
import glob
import subprocess

Welcome to JupyROOT 6.26/08


In [2]:
import hashlib
import os
username = os.environ['JUPYTERHUB_USER']
hash_object = hashlib.md5(f'{username}'.encode())
password = hash_object.hexdigest()
print(f"Username: {username}\npassword: {password}")
from dask.distributed import Client, fire_and_forget

Username: adonofrio
password: e8633ddb1c845c33b65419c5aaee0103


In [3]:
distributed = False

local_cluster = True

nmaxpartition = 35

#sanity check
if distributed == True and local_cluster == True:
    print('ERROR!!!!')

In [4]:
#cluster.close() # Nel caso sia gia' aperto

In [5]:
from dask_kubernetes.operator import KubeCluster

client = _get_global_client()

if client is not None:
    client.shutdown()


cluster = KubeCluster(
    name=f'{username}',
    namespace=f'user-{username}', 
    image='bspisso/afworker:v1',# nuova immagine !!!!!
    n_workers=10, 
    worker_command=[
        'dask-worker',
        '--name=$(DASK_WORKER_NAME)',
        '--local-directory=temp_dir', 
        '--nthreads', '10', #8 nuova, 4-5 vecchia
        '--death-timeout', '60',
        '--memory-limit','20Gi' #16 nuova, 5-6 sulla vecchia
    ],
     #port_forward_cluster_ip=True,
    resources={
        "requests": {
            "memory": "20Gi", #16 nuova, 5-6 sulla vecchia
            "cpu": "10" #8 nuova, 4-5 vecchia
        },
        "limits": {
            "memory": "20Gi", #16 nuova, 5-6 sulla vecchia
            "cpu": "10" #8 nuova, 4-5 vecchia
        }
    }
)

print(cluster.dashboard_link)

http://adonofrio-scheduler.user-adonofrio:8787/status


In [6]:
c_distributed = Client(cluster)
time.sleep(10)
c_distributed

/opt/conda/lib/python3.10/site-packages/distributed/client.py:1361: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | Client | Scheduler | Workers |
+---------+--------+-----------+---------+
| lz4     | 4.0.2  | 4.3.2     | 4.3.2   |
| msgpack | 1.0.4  | 1.0.7     | 1.0.7   |
| pandas  | 1.5.3  | 2.1.4     | 2.1.4   |
+---------+--------+-----------+---------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tcp://10.42.63.135:8786' processes=10 threads=100, memory=200.00 GiB>

In [7]:
n_workers=len(c_distributed.scheduler_info()['workers'])
inputpath = "/home/jovyan/temp_dir/dask-worker-space/"


def mv_file(file):
    path = glob.glob(inputpath + "worker-*/")[0]
    subprocess.run(['mv', path + file, inputpath])

# Funzione per distribuire e spostare i file
def distribute_and_move_files(file,c_distributed,inputpath):
    # Trova l'indice della sottostringa "mc16"
    index = file.find("mc16")   
    # Estrai la parte della stringa a partire dall'indice trovato
    rootfile = file[index:] 
    
    c_distributed.upload_file(inputpath+rootfile)
    
    # Attendi che tutti i worker siano pronti
    c_distributed.wait_for_workers(len(c_distributed.scheduler_info()['workers']))
    
    # Esegui la funzione mv_file su tutti i worker
    c_distributed.run(mv_file, rootfile)
    
    # Attendi che tutti i worker abbiano completato l'operazione di spostamento
    c_distributed.wait_for_workers(len(c_distributed.scheduler_info()['workers']))
    
    # Restituisci la lista dei file spostati
    moved_files = [inputpath + rootfile]
    return moved_files




In [8]:
def create_connection():
    """
    Setup connection to a Dask cluster. Two ingredients are needed:
    1. Creating a cluster object that represents computing resources. This can be
       done in various ways depending on the type of resources at disposal. To use
       only the local machine (e.g. your laptop), a `LocalCluster` object can be
       used. This step can be skipped if you have access to an existing Dask
       cluster; in that case, the cluster administrator should provide you with a
       URL to connect to the cluster in step 2. More options for cluster creation
       can be found in the Dask docs at
       http://distributed.dask.org/en/stable/api.html#cluster .
    2. Creating a Dask client object that connects to the cluster. This accepts
       directly the object previously created. In case the cluster was setup
       externally, you need to provide an endpoint URL to the client, e.g.
       'https://myscheduler.domain:8786'.
 
    Through Dask, you can connect to various types of cluster resources. For
    example, you can connect together a set of machines through SSH and use them
    to run your computations. This is done through the `SSHCluster` class. For
    example:
 
    ```python
    from dask.distributed import SSHCluster
    cluster = SSHCluster(
        # A list with machine host names, the first name will be used as
        # scheduler, following names will become workers.
        hosts=["machine1","machine2","machine3"],
        # A dictionary of options for each worker node, here we set the number
        # of cores to be used on each node.
        worker_options={"nprocs":4,},
    )
    ```
 
    Another common usecase is interfacing Dask to a batch system like HTCondor or
    Slurm. A separate package called dask-jobqueue (https://jobqueue.dask.org)
    extends the available Dask cluster classes to enable running Dask computations
    as batch jobs. In this case, the cluster object usually receives the parameters
    that would be written in the job description file. For example:
 
    ```python
    from dask_jobqueue import HTCondorCluster
    cluster = HTCondorCluster(
        cores=1,
        memory='2000MB',
        disk='1000MB',
    )
    # Use the scale method to send as many jobs as needed
    cluster.scale(4)
    ```
 
    In this tutorial, a cluster object is created for the local machine, using
    multiprocessing (processes=True) on 4 workers (n_workers=4) each using only
    1 core (threads_per_worker=1).
    """
    #Client locale   
    #cluster = LocalCluster(n_workers=2, threads_per_worker=1, processes=True,dashboard_address=':8786')
    cluster = LocalCluster(n_workers=2, threads_per_worker=1, processes=True, dashboard_address=':8786')
    client = Client(cluster)
    client    
    return client

#prova = create_connection()
#prova
#prova.close()

In [9]:
## Auxiliary function to count time
def timeStr(seconds):
    hours=seconds//3600
    seconds=seconds-hours*3600
    minutes=seconds//60
    seconds=seconds-minutes*60
    return f"{hours}h:{minutes}m:{int(seconds*10+0.5)/10.}s"

In [10]:
## Auxiliary function to define samples to be processed
def pathResolver(sampleName, mc):
    listFiles = []
    #campaigns = ["mc16a", "mc16d", "mc16e"]
    campaigns = ["mc16a"]
    currentDir="/home/jovyan/work/pnrr-atlas-analysis/inputs"
    if sampleName == "Data":
        currentDir+="/data"
        if mc == "mc16a":
            listFiles.append(currentDir+"/data15.root")
            listFiles.append(currentDir+"/data16.root")
        elif mc == "mc16d":
            listFiles.append(currentDir+"/data17.root")
        elif mc == "mc16e":
            listFiles.append(currentDir+"/data18.root")
        else:
            listFiles.append(currentDir+"/data15.root")
            listFiles.append(currentDir+"/data16.root")
            listFiles.append(currentDir+"/data17.root")
            listFiles.append(currentDir+"/data18.root")

    elif sampleName == "Fakes":
        if mc == "mc16a" or mc == "mc16d" or mc == "mc16e":
            listFiles.append(currentDir+"/"+mc+"/Fake_Others.root")
            listFiles.append(currentDir+"/"+mc+"/Fake_ttHnoDilep.root")
            listFiles.append(currentDir+"/"+mc+"/Fake_Others_twZ_Znunu.root")
            listFiles.append(currentDir+"/"+mc+"/Fake_VV.root")
            listFiles.append(currentDir+"/"+mc+"/Fake_Wt.root")
            listFiles.append(currentDir+"/"+mc+"/Fake_Zjets.root")
            listFiles.append(currentDir+"/"+mc+"/Fake_ttZ.root")
            listFiles.append(currentDir+"/"+mc+"/Fake_ttbar.root")
        else:
            for mcc in campaigns:
                listFiles.append(currentDir+"/"+mcc+"/Fake_Others.root")
                listFiles.append(currentDir+"/"+mcc+"/Fake_ttHnoDilep.root")
                listFiles.append(currentDir+"/"+mcc+"/Fake_Others_twZ_Znunu.root")
                listFiles.append(currentDir+"/"+mcc+"/Fake_VV.root")
                listFiles.append(currentDir+"/"+mcc+"/Fake_Wt.root")
                listFiles.append(currentDir+"/"+mcc+"/Fake_Zjets.root")
                listFiles.append(currentDir+"/"+mcc+"/Fake_ttZ.root")
                listFiles.append(currentDir+"/"+mcc+"/Fake_ttbar.root")

        if mc == "mc16a":
            listFiles.append(currentDir+"/"+mc+"/Fake_data15.root")
            listFiles.append(currentDir+"/"+mc+"/Fake_data16.root")
        elif mc == "mc16d":
            listFiles.append(currentDir+"/"+mc+"/Fake_data17.root")
        elif mc == "mc16e":
            listFiles.append(currentDir+"/"+mc+"/Fake_data18.root")
        else:
            listFiles.append(currentDir+"/mc16a/Fake_data15.root")
            listFiles.append(currentDir+"/mc16a/Fake_data16.root")
            listFiles.append(currentDir+"/mc16d/Fake_data17.root")
            listFiles.append(currentDir+"/mc16e/Fake_data18.root")

    else:
        if sampleName=="Others":
            if mc == "mc16a" or mc == "mc16d" or mc == "mc16e":
                listFiles.append(currentDir+"/"+mc+"/Others_nottHnoDilep.root")
                listFiles.append(currentDir+"/"+mc+"/ttHnoDilep.root")
                listFiles.append(currentDir+"/"+mc+"/Others_twZ_Znunu.root")
            else:
                for mcc in campaigns:
                    listFiles.append(currentDir+"/"+mcc+"/Others_nottHnoDilep.root")
                    listFiles.append(currentDir+"/"+mcc+"/ttHnoDilep.root")
                    listFiles.append(currentDir+"/"+mcc+"/Others_twZ_Znunu.root")
        else:
            if mc == "mc16a" or mc == "mc16d" or mc == "mc16e":
                listFiles.append(currentDir+"/"+mc+"/"+sampleName+".root")
            else:
                for mcc in campaigns:
                    listFiles.append(currentDir+"/"+mcc+"/"+sampleName+".root")
    return listFiles

In [11]:
## Auxiliary function to define luminosity
def GetLumi(mc):
    lumi = 1
    if mc=="mc16a": lumi=32.9881+3.21956
    elif mc=="mc16d": lumi=44.3074
    elif mc=="mc16e": lumi=58.4501
    elif mc=="Run2": lumi=58.4501+44.3074+32.9881+3.21956
    else: lumi=-1
    return lumi

print(GetLumi("Run2"))

138.96516


In [12]:
## Auxiliary function to check if TFile(s) and TTree(s) exist
def checkTree(treeName,fileName):
    hereTree=False
    logging.getLogger('HistogramExtractor').debug("Openening filename: %s", fileName)
    f=TFile(fileName)
    if f.Get(treeName): hereTree=True
    else: logging.getLogger('HistogramExtractor').error("Unable to find treename: %s", treeName)
    f.Close()
    del f
    return hereTree

In [13]:
## Auxiliary function to append extra weights
def addWeight(oldList,newWeight):
    newList = deepcopy(oldList)
    newList.extend(newWeight)
    return newList


In [14]:
## Auxiliary function to replace a weight
def replaceWeight(oldList,oldWeight,newWeight):
    newList = deepcopy(oldList)
    newList[oldList.index(oldWeight)] = newWeight
    return newList

In [15]:
if distributed == True or local_cluster == True:
    RDF = ROOT.RDF.Experimental.Distributed.Dask.RDataFrame
else:
    RDF = ROOT.RDataFrame   

## Auxiliary function to create a RDataFrame
def getRDF(nmaxpartition, sample,c_distributed, connection, distributed,input_files,suffix=""):
    if distributed == True:
        m_client = c_distributed
        for el in input_files:
            rdSample = RDF(sample + suffix, el, npartitions=nmaxpartition,daskclient=m_client)
    
    else:
        for el in input_files:
            logging.getLogger('HistogramExtractor').info("Picking up " + sample + suffix + " TTree from ")
            tc = TChain(sample + suffix)
            if checkTree(sample+suffix, el): tc.Add(el)
            if len(tc.GetListOfFiles())==0:
                logging.getLogger('HistogramExtractor').error(f"{tc.GetName()} has no root files associated")
                sys.exit(-1)
            SetOwnership(tc, False)
            # Create an RDataFrame that will use Dask as a backend for computations
            if local_cluster ==True:
                m_client = connection
                nmaxpartition = 1
                rdSample = RDF(tc,npartitions=nmaxpartition,daskclient=m_client)                    
            else:
                rdSample = RDF(tc)
            if "Preselection" in cfg.cutsDict.keys():
                logging.getLogger('HistogramExtractor').debug(f"Detected key Preselection in cutsDic ->  applying preselection filter (be sure your preselection is loose enough wrt SR selection)")
                rdSample = rdSample.Filter(cfg.cutsDict["Preselection"],"Preselection")
                if local_cluster == False:
                    SetOwnership(rdSample,False)                    
             
    return rdSample

In [16]:
## Auxiliary function to extract histogram from the given RDF, sample, regions, systematic
def getHistos(Type,rdSample,sample,channels,cfg,systWeight="1"):
    if sample == "Data": Type = "_NONE"
    listHisto = {}
    nBins = 1
    for ch in channels:
        addCuts = []
        if sample != "Fakes" and sample != "Data": addCuts.append(cfg.PromptLep)
        if not sample == "Fakes": addCuts.append(cfg.IsoSig)
        if not sample == "Data": addCuts.append(cfg.BadEvents)
        additionalCuts="( "+" && ".join(addCuts)+" )"

        cut=cfg.cutsDict[ch]
        cut+=" && ("+additionalCuts+")"
        weights="*".join(addWeight(cfg.weights,["("+systWeight+")"]))
        if sample == "Fakes": weights += "*"+cfg.fakeWeight+"*"+str(cfg.outputLumi)
        elif sample == "Data": logging.getLogger('HistogramExtractor').info("Data, no weights to add")
        else: weights +="*"+str(cfg.outputLumi)

        if sample in cfg.normalizationFactor.keys(): weights +="*"+str(cfg.normalizationFactor[sample][0])
        histoName="h"+sample+Type+"_"+ch+"_obs_cuts"
        histo=None
        chRDF=rdSample.Filter(cut,ch+" selection").Define("TotWeight","float val="+weights+";return val;").Define("varToPlot","float val=1.; return val;")
        histo = chRDF.Histo1D(ROOT.ROOT.RDF.TH1DModel(histoName,histoName,1,0.5,1.5),"varToPlot","TotWeight").GetValue()
        logging.getLogger('HistogramExtractor').debug("Defined histo: " + histoName + " with selection --> " + cut + " and weights --> " + weights)
        histo.SetBit(False)
        if histo.GetBinContent(0)!=0:
            histo.SetBinContent(1,histo.GetBinContent(0.)+histo.GetBinContent(1.))
            histo.SetBinContent(0,0.)
        if histo.GetBinContent(nBins+1)!=0:
            histo.SetBinContent(nBins+1,histo.GetBinContent(nBins)+histo.GetBinContent(nBins+1))
            histo.SetBinContent(nBins+1,0.)
        if histo.Integral()<=0:
            for i in range(1,nBins+1):
                histo.SetBinContent(i,0.0001)
        SetOwnership(histo, False)
        listHisto[ch]=histo
    return listHisto

In [17]:
############################################################### MAIN ##### HERE EVERYTHING STARTS: LET'S DO IT!
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

## Please, read the following comments: it is a trade-off as the code is being developed. It will be removed once the final implementation will be ready!
## If "Run2" is given to the campaign string, the below configuration will run for 1 sample only, i.e. Wt, for all mc campaigns.
## Otherwise, if "mc16X", X=a||d||e, a single mc campaign will be used for all samples.
campaign = "Run2"
cfg = SetConfiguration()
cfg.outputLumi = GetLumi(campaign)
cfg.outHistoFile = "histos.root"
cfg.nomName = "_WEIGHTS"

## Monte-Carlo weights
cfg.weights = ["weight","WeightEventsbTag","WeightEventsPU"]
cfg.fakeWeight = "fakeWeight"

## Load region map with associated cuts
cList = cutList(Selection="Stop4b")
cfg.cutsDict = cList.cutsDict
cfg.IsoSig = cList.IsoSigWeight
cfg.PromptLep = cList.PromptOriginWeight
cfg.FakeLep = cList.FakeOriginWeight
cfg.BadEvents = cList.BadEvents

Channels=["Stop4b_SRA", "Stop4b_SRB"]

## SYSTEMATICS: 1-side + 2-side
listEGSymSyst = ["EG_RESOLUTION_ALL","EG_SCALE_ALL"]
listJESSymSyst = ["JET_EtaIntercalibration_NonClosure_highE","JET_EtaIntercalibration_NonClosure_negEta","JET_EtaIntercalibration_NonClosure_posEta","JET_Flavor_Response","JET_GroupedNP_1","JET_GroupedNP_2","JET_GroupedNP_3"]
listJERSymSyst = ["JET_JER_DataVsMC_MC16","JET_JER_EffectiveNP_1","JET_JER_EffectiveNP_2","JET_JER_EffectiveNP_3","JET_JER_EffectiveNP_4","JET_JER_EffectiveNP_5","JET_JER_EffectiveNP_6","JET_JER_EffectiveNP_7restTerm"]
listMUONSymSyst = ["MUON_ID","MUON_MS","MUON_SAGITTA_RESBIAS","MUON_SAGITTA_RHO","MUON_SCALE"]
listMETAsymSyst = ["MET_SoftTrk_ResoPara","MET_SoftTrk_ResoPerp"]
listMETSymSyst = ["MET_SoftTrk_Scale"]
treeSystTwoSide = listEGSymSyst + listJESSymSyst + listMUONSymSyst + listMETSymSyst
treeSystOneSide = listMETAsymSyst + listJERSymSyst

## SYSTEMATICS: WEIGHT SYSTS
listWeight_EL_EFF = ["WeightEventselSF_EL_EFF_ChargeIDSel_TOTAL_1NPCOR_PLUS_UNCOR","WeightEventselSF_EL_EFF_ID_TOTAL_1NPCOR_PLUS_UNCOR","WeightEventselSF_EL_EFF_Iso_TOTAL_1NPCOR_PLUS_UNCOR","WeightEventselSF_EL_EFF_Reco_TOTAL_1NPCOR_PLUS_UNCOR"]
listWeight_MUON_EFF = ["WeightEventsmuSF_MUON_EFF_BADMUON_STAT","WeightEventsmuSF_MUON_EFF_BADMUON_SYS","WeightEventsmuSF_MUON_EFF_ISO_STAT","WeightEventsmuSF_MUON_EFF_ISO_SYS","WeightEventsmuSF_MUON_EFF_RECO_STAT","WeightEventsmuSF_MUON_EFF_RECO_STAT_LOWPT","WeightEventsmuSF_MUON_EFF_RECO_SYS","WeightEventsmuSF_MUON_EFF_RECO_SYS_LOWPT","WeightEventsmuSF_MUON_EFF_TTVA_STAT","WeightEventsmuSF_MUON_EFF_TTVA_SYS"]
listWeight_bTag = ["WeightEventsbTag_FT_EFF_B_systematics","WeightEventsbTag_FT_EFF_C_systematics","WeightEventsbTag_FT_EFF_Light_systematics","WeightEventsbTag_FT_EFF_extrapolation","WeightEventsbTag_FT_EFF_extrapolation_from_charm"]
listWeight_JVT = ["WeightEventsJVT_JET_JvtEfficiency"]
listWeight_PU = ["WeightEventsPU_PRW_DATASF"]
weightSyst = listWeight_EL_EFF + listWeight_MUON_EFF + listWeight_bTag + listWeight_JVT + listWeight_PU

## List of samples: the following if-clause is a hack to quickly test changes on a single sample. See comment above, at the beginning of the workflow
bkgSampleList=["Wt"]
if campaign == "mc16a" or campaign == "mc16d" or campaign == "mc16e":
    bkgSampleList.append("ttbar")
    bkgSampleList.append("Zjets")
    bkgSampleList.append("VV")
    bkgSampleList.append("ttZ")
    bkgSampleList.append("Others")
leftSampleList=["Fakes","Data"]
cfg.normalizationFactor = {"VV":[1.00,0.25],"ttbar":[0.90,0.12]}


# Inizia a misurare il tempo
start_time = time.time()

connection = create_connection()


## Loop to extract nominal yields
for sample in bkgSampleList:
    StartTotTime=time.time()
    StarTime = time.time()
    
    input_files = pathResolver(sample,campaign)  
    if distributed == True:
        copied_input_files = []
        for file in input_files:
            input_file = distribute_and_move_files(file,c_distributed,inputpath)
            copied_input_files.append(input_file)
        input_files = copied_input_files
        
    nomRDF = getRDF(nmaxpartition,sample,c_distributed,connection,distributed,input_files,cfg.nomName)   
    nominalHisto = getHistos("Nom",nomRDF,sample,Channels,cfg)
    for hist in nominalHisto.keys(): cfg.hists[nominalHisto[hist].GetName()]=nominalHisto[hist]
    logging.getLogger('HistogramExtractor').info('Sample: ' + sample + ' -> Produced nominal histos in ' + timeStr(time.time()-StarTime))
    
    '''
    ## Loop to extract 1-side systematics
    for syst in treeSystOneSide:
        StarTime = time.time()
        nameSystUp = "_"+syst+"__1up" if not syst=="MET_SoftTrk_ResoPara" and not syst=="MET_SoftTrk_ResoPerp" else "_"+syst
        input_files = pathResolver(sample,campaign)
        if distributed == True:
            copied_input_files = []
            for file in input_files:
                input_file = distribute_and_move_files(file,c_distributed,inputpath)
                copied_input_files.append(input_file)
            input_files = copied_input_files
        systUpRDF=getRDF(nmaxpartition,sample,c_distributed,connection,distributed,input_files,nameSystUp)
        systUpHisto=getHistos(syst+"High",systUpRDF,sample,Channels,cfg)
        for uphist in systUpHisto.keys(): cfg.hists[systUpHisto[uphist].GetName()]=systUpHisto[uphist]
        for ch in Channels:
            ## Symmetrize systematic in each channel and introduce down variation
            logging.getLogger('HistogramExtractor').info("Channel: " + ch + " --- Symmetrize 1-side SYST: " + syst)
            hNomName="h"+sample+"Nom_"+ch+"_obs_cuts"
            hUpName="h"+sample+syst+"High_"+ch+"_obs_cuts"
            hDownName="h"+sample+syst+"Low_"+ch+"_obs_cuts"
            hDown = cfg.hists[hNomName].Clone(hDownName)
            hDown.Scale(2.0)
            hDown.Add(cfg.hists[hUpName], -1.0)
        for iBin in range(1, hDown.GetNbinsX()+1):
            binVal = hDown.GetBinContent(iBin)
            if binVal < 0.: hDown.SetBinContent(iBin, 0.)
        SetOwnership(hDown, False)
        cfg.hists[hDown.GetName()]=hDown
    logging.getLogger('HistogramExtractor').info("- Produced variation: "+syst+" in "+timeStr(time.time()-StarTime))

    
    ## Loop to extract 2-side systematics
    for syst in treeSystTwoSide:
        StarTime = time.time()
        nameSystUp = "_"+syst+"__1up" if not syst=="MET_SoftTrk_Scale" else "_"+syst+"Up"
        nameSystDown = "_"+syst+"__1down" if not syst=="MET_SoftTrk_Scale" else "_"+syst+"Down"
        input_files = pathResolver(sample,campaign)
        if distributed == True:
            copied_input_files = []
            for file in input_files:
                input_file = distribute_and_move_files(file,c_distributed,inputpath)
                copied_input_files.append(input_file)
            input_files = copied_input_files
        systUpRDF  =getRDF(nmaxpartition,sample,c_distributed,connection,distributed,input_files,nameSystUp)
        systUpHisto=getHistos(syst+"High",systUpRDF,sample,Channels,cfg)
        systDownRDF=getRDF(nmaxpartition,sample,c_distributed,connection,distributed,input_files,nameSystDown)
        systDownHisto=getHistos(syst+"Low",systDownRDF,sample,Channels,cfg)
        for uphist in systUpHisto.keys(): cfg.hists[systUpHisto[uphist].GetName()]=systUpHisto[uphist]
        for downhist in systDownHisto.keys(): cfg.hists[systDownHisto[downhist].GetName()]=systDownHisto[downhist]
        logging.getLogger('HistogramExtractor').info("- Produced variation: "+syst+" in "+timeStr(time.time()-StarTime))
    
    ## Loop to extract weight systematics
    for syst in weightSyst:
        StarTime = time.time()
        systUpHisto=None
        systDownHisto=None
        if not syst=="WeightEventsPU_PRW_DATASF":
            systUpHisto=getHistos(syst+"High",nomRDF,sample,Channels,cfg,systWeight=syst+"__1up/"+syst.split("_")[0])
            systDownHisto=getHistos(syst+"Low",nomRDF,sample,Channels,cfg,systWeight=syst+"__1down/"+syst.split("_")[0])
        else:
            weightPUNom=deepcopy(cfg.weights)
            weightPUUp=replaceWeight(weightPUNom,"WeightEventsPU",syst+"__1up")
            weightPUDown=replaceWeight(weightPUNom,"WeightEventsPU",syst+"__1down")
            cfg.weights=weightPUUp
            systUpHisto=getHistos(syst+"High",nomRDF,sample,Channels,cfg)
            cfg.weights=weightPUDown
            systDownHisto=getHistos(syst+"Low",nomRDF,sample,Channels,cfg)
            cfg.weights=weightPUNom
        for uphist in systUpHisto.keys(): cfg.hists[systUpHisto[uphist].GetName()]=systUpHisto[uphist]
        for downhist in systDownHisto.keys(): cfg.hists[systDownHisto[downhist].GetName()]=systDownHisto[downhist]
        logging.getLogger('HistogramExtractor').info("- Produced variation: "+syst+" in "+timeStr(time.time()-StarTime))
    '''
    
## Finally, extract fake and data nominal yields
#for sample in leftSampleList:
#    StarTime = time.time()
#    input_files = pathResolver(sample,campaign)
#    systNONE = "_NONE" if sample == "Data" else cfg.nomName
#    nomRDF = getRDF(sample,input_files,systNONE)
#    nominalHisto = getHistos("Nom",nomRDF,sample,Channels,cfg)
#    for hist in nominalHisto.keys(): cfg.hists[nominalHisto[hist].GetName()]=nominalHisto[hist]
#    logging.getLogger('HistogramExtractor').info("Sample: " + sample + " -> Produced nominal histos in " + timeStr(time.time()-StarTime))




# Save all histograms
cfg.writeFiles()

# Termina la misurazione del tempo
end_time = time.time()

# Calcola il tempo trascorso
elapsed_time = end_time - start_time

# Stampa il risultato
print("Tempo impiegato in secondi: ", elapsed_time)   


INFO:HistogramExtractor:Picking up Wt_WEIGHTS TTree from 
INFO:HistogramExtractor:Sample: Wt -> Produced nominal histos in 0.0h:0.0m:13.5s
Storing histograms in file: histos.root
Done building file with histograms
Tempo impiegato in secondi:  14.609896898269653
